In [1]:
import os                                                                          
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"                                       
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import tensorflow as tf
session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.05, allow_growth=True), allow_soft_placement=True, log_device_placement=False)
sess = tf.Session(config=session_config)

from tensorflow.contrib.slim.nets import resnet_v2, resnet_utils
import tensorflow as tf
from tensorflow.contrib import layers as layers_lib
from tensorflow.python.ops import variable_scope
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.contrib import slim
from tensorflow.nn import ctc_loss, conv2d
import numpy as np
resnet_v2_block = resnet_v2.resnet_v2_block
resnet_v2 = resnet_v2.resnet_v2

import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
VAL_SIZE = 2048

In [3]:
def resnet_v2_26_base(inputs,
                 num_classes=None,
                 is_training=True, # True - due to update batchnorm layers
                 global_pool=False,
                 output_stride=1, # effective stride 
                 reuse=None,
                 include_root_block=False, #first conv layer. Removed due to max pool supression. We need large receprive field
                 scope='resnet_v2_26'):
  
    """
    Tensorflow resnet_v2 use only bottleneck blocks (consist of 3 layers).
    Thus, this resnet layer model consist of 26 layers.
    I put stride = 2 on each block due to increase receptive field.

    """
    blocks = [
      resnet_v2_block('block1', base_depth=64, num_units=2, stride=2),
      resnet_v2_block('block2', base_depth=128, num_units=2, stride=2),
      resnet_v2_block('block3', base_depth=256, num_units=2, stride=2),
      resnet_v2_block('block4', base_depth=512, num_units=2, stride=2),
    ]
    return resnet_v2(
      inputs,
      blocks,
      num_classes,
      is_training,
      global_pool,
      output_stride,
      include_root_block,
      reuse=reuse,
      scope=scope)

def make_ocr_net(inputs, num_classes, is_training=True):
    '''
    Creates neural network graph.
    Image width halved and it's define timestamps width (feature sequence length) 
    No activation after output (no softmax), due to it's presence at ctc_loss() and beam_search().
    After resnet head features are resized to be [batch,1,width,channel], and after that goes 1x1 conv 
    to make anology of dense connaction for each timestamp.
    
    input: batch of images
    output: tensor of size [batch, time_stamps_width, num_classes]
    '''
    with tf.variable_scope('resnet_base', values=[inputs]) as sc:
        with slim.arg_scope([slim.conv2d],
                              activation_fn=None, normalizer_fn=None):
            net = resnet_utils.conv2d_same(inputs, 64, 7, stride=2, scope='conv1') #root conv for resnet
            #net = slim.max_pool2d(net, [3, 3], stride=2, scope='pool1') # due to enlarge of receptive field
            net = resnet_v2_26_base(net, output_stride=1, is_training = is_training)[0] # ouput is a tuple of last tensor and all tensors 
    with tf.variable_scope('class_head', values=[net]) as sc:
        net = tf.transpose(net, [0,3,1,2]) # next 4 lines due to column to channel reshape. [batch,c,h,w]
        _,c,h,_ = net.get_shape() # depth of input to conv op tensor should be static (defined)
        shape = tf.shape(net)
        net = tf.reshape(net, [shape[0], c*h, 1, shape[3]])
        net = tf.transpose(net,[0,2,3,1]) # back to [batch,h,w,c] = [batch,1,w,features*h]
        net = layers_lib.conv2d(net, num_classes, [1, 1], activation_fn=None) #CTC got softmax [batch,1,w,num_classes]
#         net = layers_lib.softmax(net)
        net = tf.squeeze(net,1) #[batch,w,num_classes]
        return net
def get_num_only(logits):
    '''logits is [time, batch, logits]
    because output of net not softmax but logit, it minimun not zero but some negative value
    and we should substract this value from not allowed chars (not num)
    '''
    mask_index = [char_to_indx[char] for char in char_to_indx if char not in numeric_chars]
    abs_logit_max = tf.reduce_max(tf.abs(logits))
    mask = [0]*logits.shape[-1]
    for indx in mask_index:
        mask[indx+1] = abs_logit_max 
    mask[0]=0
    mask[-1]=0
    mask = tf.convert_to_tensor(mask, dtype=tf.float32)
    filtered_loggits = logits-mask
#     filtered_loggits = tf.Print(filtered_loggits, [tf.argmax(filtered_loggits,-1)], summarize=100000)
    return filtered_loggits
def get_prediction(output_net, is_num, seq_len, merge_repeated=False):
    '''
    predict by using beam search
    input: output_net - logits (without softmax) of net
           seq_len - length of predicted sequence 
    '''
    net = tf.transpose(output_net, [1, 0, 2]) #transpose to [time, batch, logits]
    net = tf.cond(is_num, lambda: get_num_only(net), lambda: net)
    
    decoded, prob = tf.nn.ctc_beam_search_decoder(net, seq_len, merge_repeated=merge_repeated, beam_width=3)
#     decoded = tf.arg_max(net, -1)
    return decoded, prob


In [4]:
with open('../../data_generator/valid_charset.txt', 'r', encoding = 'utf-8') as f:
    valid_charset = f.read()
all_chars = list(valid_charset)
char_to_indx = dict(zip(all_chars,range(len(all_chars))))
num_classes = len(all_chars)

In [5]:
import json
from sklearn.model_selection import train_test_split

with open('../datasets/labeled_strings.json', 'r') as f:
    string_data = json.load(f)

def my_stratify_value(data, num_of_bins):
    '''
    :param data: is a pair sequence of image path and string
    '''
    str_lens = []
    for example in data:
        string = example['context']
        str_lens.append(len(string))
    bins = np.linspace(min(str_lens), max(str_lens)+1, num_of_bins+1)
    stratify_values = np.digitize(str_lens, bins)
    return stratify_values

def stratify_split_train_test(data, portion, func):
    '''
    :param func: function which return value (based on data example)  to be used in stratification of dataset split
    '''
    num_of_bins = 100
    while True:
        stratify_values = func(data, num_of_bins)
        try:
            split = train_test_split(data, test_size=portion, stratify=stratify_values, random_state=42)
        except ValueError:
            print('number of bins {} is too hight for stratification. Lowering to {}'.format(num_of_bins, num_of_bins-1))
            num_of_bins -= 1
            continue
        return split

# string_data_tupels = [('../datasets/'+string_data[i]['path'], string_data[i]['context'])
#                       for i in np.random.choice(range(len(string_data)), size=1024, replace=False)]
string_data_tupels = stratify_split_train_test(string_data, VAL_SIZE, func = my_stratify_value)
string_data_tupels_test = string_data_tupels[1] # test only taken
string_data_tupels_test = [('../datasets/'+i['path'], i['context'])
                      for i in string_data_tupels_test]
string_data_tupels_train = string_data_tupels[0]
string_data_tupels_train = [('../datasets/'+i['path'], i['context'])
                      for i in string_data_tupels_train]

number of bins 100 is too hight for stratification. Lowering to 99
number of bins 99 is too hight for stratification. Lowering to 98
number of bins 98 is too hight for stratification. Lowering to 97
number of bins 97 is too hight for stratification. Lowering to 96
number of bins 96 is too hight for stratification. Lowering to 95
number of bins 95 is too hight for stratification. Lowering to 94
number of bins 94 is too hight for stratification. Lowering to 93
number of bins 93 is too hight for stratification. Lowering to 92
number of bins 92 is too hight for stratification. Lowering to 91
number of bins 91 is too hight for stratification. Lowering to 90
number of bins 90 is too hight for stratification. Lowering to 89
number of bins 89 is too hight for stratification. Lowering to 88
number of bins 88 is too hight for stratification. Lowering to 87
number of bins 87 is too hight for stratification. Lowering to 86
number of bins 86 is too hight for stratification. Lowering to 85
number of

In [6]:
print(dict(zip(range(len(all_chars)), all_chars)))

{0: 'Е', 1: 'Щ', 2: 'Э', 3: 'e', 4: 'M', 5: 'S', 6: 'н', 7: 'ь', 8: 'В', 9: 'у', 10: 'I', 11: 'R', 12: '7', 13: '8', 14: '"', 15: 'й', 16: 'Ф', 17: 'u', 18: 'B', 19: 'П', 20: 'Я', 21: '=', 22: 'Ж', 23: '-', 24: 'и', 25: '№', 26: 'y', 27: 'n', 28: 'T', 29: '!', 30: 'a', 31: 'З', 32: 'ъ', 33: 'X', 34: 'O', 35: 'Ю', 36: 't', 37: 'Ч', 38: 'п', 39: '+', 40: 'v', 41: 'Г', 42: 'f', 43: 'z', 44: '&', 45: 'F', 46: 'з', 47: '.', 48: 'Б', 49: ')', 50: '3', 51: 'Y', 52: 'х', 53: '6', 54: '*', 55: 'Ё', 56: '_', 57: 'N', 58: '9', 59: 'q', 60: 'r', 61: 'K', 62: 'd', 63: '0', 64: '%', 65: 'U', 66: ',', 67: 'c', 68: 'W', 69: 'У', 70: 'р', 71: 'j', 72: 'E', 73: 'к', 74: 'g', 75: 'ш', 76: 'ж', 77: '4', 78: '1', 79: '~', 80: 'D', 81: 'Х', 82: '5', 83: 'Н', 84: 'G', 85: 'С', 86: 'Z', 87: 'k', 88: 'Л', 89: 'ё', 90: 'J', 91: 'ф', 92: 'w', 93: 'ю', 94: 'д', 95: 'л', 96: 'К', 97: 'т', 98: 'A', 99: 'Q', 100: 'м', 101: ':', 102: 'H', 103: 'ы', 104: 'е', 105: 'с', 106: 'г', 107: '(', 108: 'l', 109: 'Т', 110: '2',

In [7]:
numeric_chars = '0123456789,.'

In [8]:
class OCRModel(object):
    def __init__(self, charset, input_image_batch, sequence_labels, is_num, is_training=True, learning_rate=1e-4,
                decay_steps=2**16, decay_rate=0.9, decay_staircase=False, momentum=0.9):
        self.charset = charset
        self.num_classes = len(charset)+2 #indexing starts from one, +empty char. Yes, I know that I got 0-th rudimental output, whatever
        self.is_training = is_training
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate 
        self.decay_staircase = decay_staircase
        self.momentum = momentum
        mapping_char = tf.constant(list(charset))
        self.table = tf.contrib.lookup.index_to_string_table_from_tensor(
                    mapping_char, default_value="")
        self.is_num = is_num

        self.build_for_prediction(input_image_batch)
        
    def build_for_prediction(self, input_image_batch):
        
        self.input_image_batch = input_image_batch
        net = make_ocr_net(self.input_image_batch, self.num_classes, is_training=self.is_training)
        self.net = net
        self.prediction = get_prediction(output_net=net, is_num=self.is_num, 
                                         seq_len=[tf.shape(self.net)[1]], merge_repeated=False) # tuple(decoded, prob). decoded - list of top paths. I use top1
        pred_dense = tf.sparse_to_dense(self.prediction[0][0].indices, self.prediction[0][0].dense_shape, 
                                               self.prediction[0][0].values)
#         pred_dense = self.prediction
        self.pred_dense = pred_dense
        self.prediction_string = tf.reduce_join(self.table.lookup(tf.cast(pred_dense, tf.int64)-1))
        

In [9]:
import json
from skimage import io
from skimage.transform import rescale
from skimage.color import rgb2gray
def resize_height(image, max_dim):
    h, w = image.shape
    scale_factor = max_dim/h
    output_image = rescale(image.copy(), scale = scale_factor, order = 3)
    return output_image

In [10]:
graph = tf.Graph()
with graph.as_default():

    features = tf.placeholder(tf.float32, shape = [None, 32, None, 1])
    is_num = tf.placeholder(tf.bool, shape = [])
    model = OCRModel(charset = all_chars, input_image_batch=features, 
                     sequence_labels=None, is_training=False, is_num=is_num)
    init = tf.global_variables_initializer()
    table_init = tf.tables_initializer()
    saver = tf.train.Saver()

In [11]:
from lev_dist import distance

In [12]:
def is_num_string(string):
    return len(set(string).intersection(set(numeric_chars))) == len(set(string))

In [13]:
%%time
LOG_DIR = 'log_real_data_learn_2/'
with tf.Session(graph=graph) as sess:
#     run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#     run_metadata = tf.RunMetadata()
    sess.run([init, table_init])
    try:
        saver.restore(sess, LOG_DIR + "model_lev_dist_0.022212269778200858.ckpt")
    except:
        print('cant restore')
#     with open('../datasets/labeled_strings.json', 'r') as f:
#         string_data = json.load(f)
    match = 0
    lev_dist = 0
    for example in string_data_tupels_test:
        image = io.imread(example[0])
        image = rgb2gray(image)
        image = resize_height(image, 32)
#         image = image/255 #as image scaled to [0,1]
        image = image - 0.5
        string = example[1]
        pred_string = sess.run([model.prediction_string], feed_dict={features: image[np.newaxis,...,np.newaxis],
                                                                 is_num: is_num_string(string)})
        pred_string = pred_string[0].decode('utf-8')
        print(pred_string.upper(), string)
        lev_dist += distance(pred_string.upper(), string) 
        if pred_string.upper() == string:
            match+=1

INFO:tensorflow:Restoring parameters from log_real_data_learn_2/model_lev_dist_0.022212269778200858.ckpt


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


1139 1139
DATA :13-02-2018 DATA :13-02-2018
2010001277231 2010001277231
РУБ РУБ
73 73
1ШТ. 1 ШТ.
625 625
ВЕРМУТ ЧИНЗАНО БЬЯНКО 0,5Л ВЕРМУТ ЧИНЗАНО БЬЯНКО 0,5Л
DATA :11-02-2018 DATA :11-02-2018
100 ГР = 149 100 ГР = 149
Л У Ч Ш АЯ Л У Ч Ш АЯ
1ШТ. 1ШТ.
-25% -25%
МАЛИНА 400Г МАЛИНА 400Г
РARМАLАТ PARMALAT
01.03.2018 01.03.2018
ОТБОРНОЕЗ,4-4,5% 930 МЛ. ОТБОРНОЕЗ4-4,5% 930 МЛ.
20 20
РУБ РУБ
1 КГ =106,89РУБ 1 КГ =106,89РУБ
64 64
76 76
45 45
74 74
12 12
-13% -13%
100 ГР = 158 100 ГР = 158
-43% -43%
ЛУКАШОВ Д.В. ЛУКАШОВ Д.В.
РУБ РУБ
ПРОСТОКВАШИНО ПРОСТОКВАШИНО
ЧУДОДЕСЕРТ ТВОРОЖНЫЙ 4% ЧУДО ДЕСЕРТ ТВОРОЖНЫЙ 4%
1000112678 1000112678.
15 15
99 99
БАБЛМИНТ УЛ 10 13,5Г БАБЛМИНТ УП 10*13,6Г
1894700008 1894700008
С ПЕЧЕНЬЕМ МОЛ 87Г С ПЕЧЕНЬЕМ МОЛ 87Г
1КГ 1КГ
159 159
83 83
АКЦИЯТ АКЦИЯ!
28 28
1000169052 1000169052
1 КГ =123,79РУБ 1 КГ =123,79РУБ
DATA :13-02-2018 DATA :13-02-2018
79 79
РУБ РУБ
ВЯЛЕН ФЛАНДЕРР 200Г ВЯЛЕН ФЛАНДЕРР 200Г
43 43
37 37
26.02.2018 26.02.2018
30 30
DATA :23-02-2018 DATA :23-02-201

135Г 135Г
1 КГ=78.48РУБ 1 КГ=78.48РУБ
1000149242 1000149242
ПАСТА ЗАРА ПАСТА ЗАРА
99 99
85 85
99 99
-18% -18%
41 41
DATA :20-02-2018 DATA :20-02-2018
САЛАТ ЛИСТОВОЙ САЛАТ ЛИСТОВОЙ
М&МS ДРАЖЕ С ШОКОЛАДОМ M&MS ДРАЖЕ С ШОКОЛАДОМ
РATA :25-02-2018 ATA :25-02-2018
БАНАН-ЯБЛ. 4,2% 100Г БАНАН-ЯБЛ.4,2% 100Г
20 20
90 90
99 99
99 99
РУБ РУБ
90 90
1000184685 1000184685
DATA :07-02-2018 DATA :07-02-2018
DATA :30-05-2017 DATA :30-05-2017
1КГ = 3624 1 КГ = 362
ЦЕНА ЦЕНА
1 Л =72,18РУБ 1 Л =72,18РУБ
РУБ РУБ
30 30
ПАСТ.МОЛОКО ВЕС.МОЛ ОТБ 930 ПАСТ. МОЛОКО ВЕС.МОЛ ОТБ 930
31 31
99 99
ПРОДУКТ ТВОРОЖНЫИ ДАНИССИМО ПРОДУКТ ТВОРОЖНЫИ ДАНИССИМО
08 08
СВИНКА ПЕППА 70Г СВИНКА ПЕППА 70Г
32 32
С МАРМЕЛАД НАЧ 330Г С МАРМЕЛАД НАЧ 330Г
1 КГ =56.90РУБ 1 КГ =56.90РУБ
41 41
СЛИВКИ 350ГР 10% КОМБИФИТ ПРО СЛИВКИ 350ГР 10% КОМБИФИТ ПРО
DATA :23-02-2018 DATA :23-02-2018
90 90
90 90
1ШТ. 1ШТ.
88 88
1 КГ =56,90РУБ 1 КГ =56,90РУБ
DATA :24-02-2018 DATA :24-02-2018
100 ГР = 61.53 100 ГР = 61.53
74 74
АКЦИЯ! АКЦИЯ!
79 79
1ШТ. 1 ШТ

1 КГ =37,80РУБ 1 КГ =37,80РУБ
ЧЕСНОК ФИШКА 130Г ЧЕСНОК ФИШКА 130Г
РУБ РУБ
2400025004 2400025004
МАЛЬТЛЬЯТИ МАЛЬТАЛЬЯТИ
1 КГ =163,75РУБ 1 КГ =163,75РУБ
КИЛЬКА БАЛИЙСКАЯ В КИЛЬКА БАЛТИЙСКАЯ В
100 ГР = 130.17 100 ГР = 130.17
144 144
БАНАН 4,2% 100Г БАНАН 4,2% 100Г
00 00
100 ГР = 2869 100 ГР = 28699
19 19
ТВОРОЖОК КЛУБНИКА 100Г ЧУДО ТВОРОЖОК КЛУБНИКА 100Г ЧУДО
99 99
78 78
87 87
1 Л =1849,87РУБ 1 Л =1849,87РУБ
84 84
DATA :26-02-2018 DATA :26-02-2018
20 20
83 83
47 47
1000171944 1000171944
DATA :22-2-2018 DATA :22-02-2018
РУБ РУБ
РУБ РУБ
72 72
1 КГ = 342 1 КГ = 342
СРЕДСТВО ДЛЯ СТИРКИ ЖИДКОЕ СРЕДСТВО ДЛЯ СТИРКИ ЖИДКОЕ
РУБ РУБ
РУБ РУБ
14.02.2018 14.02.2018
DATA :Ф3-10-201 DATA 07-10-2017
51 51
ШОКОЛАД 5УШЕРОН ГОР ШОКОЛАД БУШЕРОН ГОР
DATA :01-03-2018 DATA :01-03-2018
М/УП(АНГСТРЕМ ТРЕЙДИНГ) М/УП(АНГСТРЕМ ТРЕЙДИНГ)
1КГ 1 КГ
81 81
ЦЕНА Р/ШТ ЦЕНА Р/ШТ
90 90
РУБ. РУБ.
ИВАНОВА ПАСЕКА МЁД НАТУР ИЗАНОВА ПАСЕКА МЁД НАТУР
ЦЕЖА Р/ШТ ЦЕНА Р/ШТ
NESCAFE GOLD ЕRGОS КОФЕ М/У NESCAFE GOLD ERGOS КОФЕ М/У
42 42

8999 8999
74 74
ПЬЮР ВАЙТ МЯТА 54Г ПЬЮР ВАЙТ МЯТА 54Г
DATA :16-02-2018 DATA :16-02-2018
JАCOВS MONARCН КОФЕ СУБЛ. С/Б JACOBS MONARCH КОФЕ СУБЛ. С/Б
200МЛ Т/ П (ВБД) :18 200МЛ Т/ П (ВБД) :18
07 07
РУБ РУБ
99 99
99 99
ЦЕНА Р/ШТ ЦЕНА Р/ШТ
1 КГ = 11999 1 КГ = 1199
80 80
ШОКОЛАД ГЛОБУС ВИТА ШОКОЛАД ГЛОБУС ВИТА
55 55
109 109
99 99
СВВЕЖЖИЙ СВЕЖИЙ
ROМAНCE СAШЕШЕЛК ROMANCE САШЕ"ШЕЛК"
16 16
90 90
00 00
163 163
27.02.2018 27.02.2018
СМЕТАНА 20% 330Г ДВД СМЕТАНА 20% 330Г ДВД
1000173873000 1000173873000
ПИКЕР КАЛУА 0,7Л США ЛИКЕР КАЛУА 0,7Л США
1 ШТ. 1 ШТ.
-41% -41%
ЦЕН А ЦЕН А
49 49
46 46
92990 92990
1 ШТ. 1 ШТ.
ПРО СМ~ТАНА 10% 315Г ПРО СМ-ТАНА 10% 315Г
18 1 8
100 ГР = 351 100 ГР = 3541
460441011186 4602441011186
СТАРАЯ СТАРАЯ
99 99
100 ГР = 14 100 ГР = 1444
20 20
DATA :12-02-2018 DATA :12-02-2018
ОЛОКО 3,2% 0,5Л ТЕМА ОЛОКО 3,2% 0,5Л ТЕМА
ПЭТ(ЮНИМИЛК) ПЭТ(ЮНИМИЛК)
1 КГ 1 КГ
ПОЛЕТ ДРАКОНА 90Г ПОЛЕТ ДРАКОНА 90Г
71 71
1 КГ =157,66РУБ 1 КГ = 157,66 РУБ
60 60
1КГ 1КГ
-29% -29%
01 01
ВАШ ВЫБОР ЯБЛОКО 3

РУБ РУБ
2 В 1 100Г 2В 1 100Г
СВЕЖАЯ СВЕЖАЯ
С КИД КА СКИД КА
DATA :24-02-2018 DATA :24-02-2018
100 ГР = 9187 100 ГР = 9187
СУФЛЕ ВЕСЕЛОЕ НА СУФЛЕ ВЕСЕЛОЕ НА
КИТАЙСКАЯ КИТАЙСКАЯ
399 399
34 34
BREESAL АРОМАТИЧЕСКИЕ ГЕЛЕВЫЕ BREESAL АРОМАТИЧЕСКИЕ ГЕЛЕВЫЕ
РУБ РУБ
С МИНДАЛЕМ Ж/Б 300Г С МИНДАЛЕМ Ж/Б 300Г
ЦЕНА ЦЕНА
90 90
ДЕСЕРТ БОНЖУР ВКУС ДЕСЕРТ БОНЖУР ВКУС
ТВОРОГ 0,2% 170Г ДВД ТВОРОГ 0,2% 170Г ДВД
ПЕЧЕНЬЕ ПИКАП СЕНДВ ПЕЧЕНЬЕ ПИКАП СЕНДВ
DATA :13-05-2017 DATA :13-05-1017
3,2% 950МЛ ТБА(ЮНИМИЛК) 3,2% 950МЛ ТБА(ЮНИМИЛК)
82 82
30 30
45 45
219 219
ТБА 750Г ТБА 750Г
99 99
РУБ РУБ
1ШТ. 1 ШТ.
1 Л =36,67РУБ 1 Л =36,67РУБ
DATA :28-10-2017 DATA :28-10-2017
31 31
48 48
70 70
РУБ РУБ
3083680616904 3083680616904
РУБ РУБ
РУБ РУБ
НОВИНКА НОВИНКА
DATA :24-02-2018 DATA :24-02-2018
250Г ГБ 250Г ГБ
АПЕЛЬСИН 300Г АПЕЛЬСИН 300Г
АГУША ЙОГУРТ ВЯЗКИЙ АГУША ЙОГУРТ ВЯЗКИЙ
0,7Л 0,7Л
1 ШТ. 1 ШТ.
94898000 194898000
47 47
50Г(ТУРШУ) 50Г(ТУРШУ)
1 Л =822,06РУБ 1 Л=830,06РУБ
90 90
1 Л =71,51РУБ 1 Л =71,51РУБ
РУБ. РУБ.
49 49
М

ЗЕЛЕНЫЙ 425Г Ж/ Б (ДЖЕНЕРАЛ ЗЕЛЕНЫЙ 425Г Ж/ Б (ДЖЕНЕРАЛ
39 39
РУБ РУБ
1КГ 1КГ
ФАСОВАННЫЙ СЕТКА ФАСОВАННЫЙ СЕТКА
ДОМИК В ДЕРЕВНЕ СЛИВКИ ДОМИК В ДЕРЕВНЕ СЛИВКИ
93 93
53 53
99 99
8990 8990
ОВОЩАМИ 0,53КГ (АСТР НАТ ПРОД) ОВОЩАМИ 0,53КГ (АСТР НАТ ПРОД)
95 95
1 КГ =224,06РУБ 1 КГ=224,06РУБ
1 Л =1933,94РУБ 1 Л =1933,04РУБ
109 109
99 99
DATA :13-02-2018 DATA :13-02-2018
99 99
РУБ РУБ
ИМПЕРИИ 25 ПАК/ 2Г :27 ИМПЕРИИ 25 ПАК/2Г :27
ЧИЩЕННЫЕ 100Г ЧИЩЕННЫЕ 100Г
ВИНОГРАД ВИНОГРАД
РУБ РУБ
01 01
CPU times: user 30 s, sys: 1min 10s, total: 1min 40s
Wall time: 41.8 s


CPU times: user 5min 4s, sys: 3min 54s, total: 8min 59s <br>
Wall time: 5min 19s

In [14]:
print('Levinstain distance: ', lev_dist/len(string_data_tupels_test))
print('Accurancy: ', match/len(string_data_tupels_test))

Levinstain distance:  0.02091849310934511
Accurancy:  0.84521484375


model_lev_dist_0.026286025077569927.ckpt <br>
Levinstain distance:  0.024396842622747453 <br>
Accurancy:  0.82568359375

model_lev_dist_0.03035412592912709.ckpt <br>
Levinstain distance:  0.028162182910711254 <br>
Accurancy:  0.81005859375

CPU times: user 25.4 s, sys: 51.8 s, total: 1min 17s <br>
0.03089280631506283 <br>
Wall time: 31.8 s <br>
Levinstain distance:  0.02722495414765611 <br>
Accurancy:  0.814453125

with beamsearch <br>
log_real_data_learn_2/model_lev_dist_0.03089280631506283.ckpt <br>
CPU times: user 5min 5s, sys: 3min 57s, total: 9min 3s <br>
Wall time: 5min 24s <br>
Levinstain distance:  0.027249368210156107 <br>
Accurancy:  0.81396484375

log/' + "model_lev_dist_0.2719941635845625.ckpt <br>
Levinstain distance:  0.2582129033096021 <br>
Accurancy:  0.3056640625 <br>